[View in Colaboratory](https://colab.research.google.com/github/bell-one/tf_colab/blob/master/YELLOW_LSTM_tf.ipynb)

In [0]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

In [0]:
sample = ("Look at the stars Look how they shine for you And everything you do"
"Yeah they were all yellow I came along I wrote a song for you"
"And all the things you do And it was called Yellow"
"So then I took my turn Oh what a thing to have done And it was all yellow")
char_set = list(set(sample))
char_dic = {w: i for i, w in enumerate(char_set)}

In [6]:
# hyper parameters
dic_size = len(char_set)  # RNN input size (one hot size)
hidden_size = len(char_set)  # RNN output size
num_classes = len(char_set)  # final output size (RNN or softmax, etc.)
learning_rate = 0.05
seq_length = 10  # number of lstm rollings (unit #)

dataX = []
dataY = []

for i in range(0, len(sample) - seq_length):
  x_str = sample[i:i + seq_length]
  y_str = sample[i+1:i+seq_length+1]
  print(i, x_str, '->', y_str)
  
  x = [char_dic[c] for c in x_str]
  y = [char_dic[c] for c in x_str]
  
  dataX.append(x)
  dataY.append(y)


batch_size = len(dataX)

0 Look at th -> ook at the
1 ook at the -> ok at the 
2 ok at the  -> k at the s
3 k at the s ->  at the st
4  at the st -> at the sta
5 at the sta -> t the star
6 t the star ->  the stars
7  the stars -> the stars 
8 the stars  -> he stars L
9 he stars L -> e stars Lo
10 e stars Lo ->  stars Loo
11  stars Loo -> stars Look
12 stars Look -> tars Look 
13 tars Look  -> ars Look h
14 ars Look h -> rs Look ho
15 rs Look ho -> s Look how
16 s Look how ->  Look how 
17  Look how  -> Look how t
18 Look how t -> ook how th
19 ook how th -> ok how the
20 ok how the -> k how they
21 k how they ->  how they 
22  how they  -> how they s
23 how they s -> ow they sh
24 ow they sh -> w they shi
25 w they shi ->  they shin
26  they shin -> they shine
27 they shine -> hey shine 
28 hey shine  -> ey shine f
29 ey shine f -> y shine fo
30 y shine fo ->  shine for
31  shine for -> shine for 
32 shine for  -> hine for y
33 hine for y -> ine for yo
34 ine for yo -> ne for you
35 ne for you -> e for you 
36

In [0]:
X = tf.placeholder(tf.int32, [None, seq_length])
Y = tf.placeholder(tf.int32, [None, seq_length])
X_one_hot = tf.one_hot(X, num_classes)

In [0]:
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, X_one_hot, initial_state = initial_state, dtype=tf.float32)
weights = tf.ones([batch_size,seq_length])
seq_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets =Y, weights=weights)

loss = tf.reduce_mean(seq_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
prediction = tf.argmax(outputs, axis=2)

In [14]:
sess.run(tf.global_variables_initializer())
for i in range(500):
    _, l, results = sess.run(
        [train, loss, outputs], feed_dict={X: dataX, Y: dataY})
    for j, result in enumerate(results):
        index = np.argmax(result, axis=1)

# Let's print the last char of each result to check it works
results = sess.run(outputs, feed_dict={X: dataX})
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:  # print all for the first result to make a sentence
        print(''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]], end='')

Looo at the stars  ooo how they shine for you dnd e erything you dooeah they were all yellow I oaee along I wrote a song for youond all the things you do nnd it was aalled eellowSo then I tooo oy turn oh what a thing to haee done nnd it was all yello